In [1]:
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

with open('titanic_step4_importance_train.pickle', 'rb') as pickle_filename:
    train_importance = pickle.load(pickle_filename)
with open('titanic_step4_importance_test.pickle', 'rb') as pickle_filename:
    test_importance = pickle.load(pickle_filename)
with open('titanic_step4_importance_train_y.pickle', 'rb') as pickle_filename:
    train_answer = pickle.load(pickle_filename)

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import numpy as np
from bayes_opt import BayesianOptimization

In [14]:
pbounds = {
    'learning_rate':(0.01, 0.5),
    'n_estimators': (100, 1000),
    'max_depth' : (3, 10),
    'min_child_weight': (0, 10),
    'subsample': (0.5, 1.0),
    'colsample_bytree':(0.5, 1.0),
    'gamma':(0, 5)
}

In [19]:
def xgboost_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight, subsample, colsample_bytree, gamma):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBClassifier(
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        random_state=1,
        eval_metric='logloss',
    )
    return np.mean(cross_val_score(clf, train_importance, train_answer, cv=5, scoring='accuracy'))

In [20]:
optimizer=BayesianOptimization(f=xgboost_hyper_param, pbounds=pbounds, random_state=1)
optimizer.maximize(init_points=10, n_iter=100)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.8126    | 0.7085    | 3.602     | 0.01006   | 5.116     | 1.468     | 183.1     | 0.5931    |
| 2         | 0.8283    | 0.6728    | 1.984     | 0.274     | 5.934     | 6.852     | 284.0     | 0.9391    |
| 3         | 0.8361    | 0.5137    | 3.352     | 0.2145    | 6.911     | 1.404     | 278.3     | 0.9004    |
| 4         | 0.807     | 0.9841    | 1.567     | 0.3492    | 9.135     | 8.946     | 176.5     | 0.5195    |
| 5         | 0.8048    | 0.5849    | 4.391     | 0.05819   | 5.948     | 9.579     | 579.8     | 0.8459    |
| 6         | 0.8115    | 0.6578    | 3.433     | 0.419     | 3.128     | 7.501     | 990.0     | 0.8741    |
| 7         | 0.8047    | 0.6402    | 3.946     | 0.06058   | 6.135     | 9.086     | 364.3     | 0.6439    |
| 8       

In [21]:
optimizer.max

{'target': 0.8439897056054233,
 'params': {'colsample_bytree': 0.8731850168106186,
  'gamma': 2.6215904163454007,
  'learning_rate': 0.3011825987454647,
  'max_depth': 9.30914843329363,
  'min_child_weight': 1.2048301359087699,
  'n_estimators': 467.48342731051787,
  'subsample': 0.829554081618765}}